In [2]:
# Reddit Gold Layer - Monthly Aggregation
# Combines submissions and comments into monthly parquet files with clean schemas
import sys
from pathlib import Path
from datetime import datetime
import pandas as pd
from tqdm import tqdm

workspace_root = Path.cwd()
while not (workspace_root / '.git').exists() and workspace_root != workspace_root.parent:
    workspace_root = workspace_root.parent

print(f"Gold layer processing started: {datetime.now().isoformat()}")
print(f"Workspace: {workspace_root}")

Gold layer processing started: 2025-12-21T09:05:35.215081
Workspace: /Users/stahlma/Desktop/01_Studium/11_Thesis/Data_Experiment


## 1. Setup Paths

Define input (silver) and output (gold) directories.

In [3]:
# Input paths
comments_silver_dir = workspace_root / 'data/01_corpus/01_silver/reddit/politosphere'
submissions_silver_dir = workspace_root / 'data/01_corpus/01_silver/reddit/submissions'

# Output paths
gold_submissions_dir = workspace_root / 'data/01_corpus/02_gold/reddit/submissions'
gold_comments_dir = workspace_root / 'data/01_corpus/02_gold/reddit/comments'

gold_submissions_dir.mkdir(parents=True, exist_ok=True)
gold_comments_dir.mkdir(parents=True, exist_ok=True)

print(f"Comments silver: {comments_silver_dir.relative_to(workspace_root)}")
print(f"Submissions silver: {submissions_silver_dir.relative_to(workspace_root)}")
print(f"\nGold submissions: {gold_submissions_dir.relative_to(workspace_root)}")
print(f"Gold comments: {gold_comments_dir.relative_to(workspace_root)}")

Comments silver: data/01_corpus/01_silver/reddit/politosphere
Submissions silver: data/01_corpus/01_silver/reddit/submissions

Gold submissions: data/01_corpus/02_gold/reddit/submissions
Gold comments: data/01_corpus/02_gold/reddit/comments


## 2. Language Detection Setup

Load language detection model for comments.

In [4]:
from langdetect import detect, LangDetectException

def detect_language(text):
    """Detect language of text, return 'unknown' on failure"""
    if pd.isna(text) or not text.strip():
        return 'unknown'
    try:
        return detect(text)
    except LangDetectException:
        return 'unknown'

print("✓ Language detection ready (using langdetect library)")

✓ Language detection ready (using langdetect library)


## 3. Process Submissions by Month

Aggregate daily submission files into monthly parquet files with clean schema.

In [5]:
months = ['2016-09', '2016-10']

for month in months:
    print(f"\nProcessing submissions for {month}...")
    print("=" * 80)
    
    # Find all daily files for this month
    daily_files = sorted(submissions_silver_dir.glob(f'{month}-*.parquet'))
    print(f"Found {len(daily_files)} daily files")
    
    if not daily_files:
        print(f"⚠️  No files found for {month}")
        continue
    
    # Load and concatenate
    dfs = []
    for f in tqdm(daily_files, desc=f"Loading {month}"):
        df = pd.read_parquet(f)
        dfs.append(df)
    
    df_month = pd.concat(dfs, ignore_index=True)
    print(f"Loaded {len(df_month):,} submissions")
    
    # Apply clean schema (no author for GDPR compliance)
    df_gold = df_month[[
        'thread_id',
        'title',
        'selftext',
        'created_utc',
        'subreddit_id',
        'subreddit',
        'num_comments'
    ]].copy()
    
    # Rename thread_id to submission_id for clarity
    df_gold = df_gold.rename(columns={'thread_id': 'submission_id'})
    
    # Save monthly file
    output_file = gold_submissions_dir / f'{month}.parquet'
    df_gold.to_parquet(output_file, compression='snappy', index=False)
    
    print(f"✓ Saved: {output_file.name}")
    print(f"  Columns: {list(df_gold.columns)}")
    print(f"  Rows: {len(df_gold):,}")

print("\n" + "=" * 80)
print("✓ Submissions processing complete")


Processing submissions for 2016-09...
Found 30 daily files


Loading 2016-09: 100%|██████████| 30/30 [00:00<00:00, 67.33it/s]


Loaded 386,214 submissions
✓ Saved: 2016-09.parquet
  Columns: ['submission_id', 'title', 'selftext', 'created_utc', 'subreddit_id', 'subreddit', 'num_comments']
  Rows: 386,214

Processing submissions for 2016-10...
Found 31 daily files


Loading 2016-10: 100%|██████████| 31/31 [00:00<00:00, 62.71it/s]


Loaded 537,217 submissions
✓ Saved: 2016-10.parquet
  Columns: ['submission_id', 'title', 'selftext', 'created_utc', 'subreddit_id', 'subreddit', 'num_comments']
  Rows: 537,217

✓ Submissions processing complete


## 4. Process Comments by Month

Aggregate daily comment files into monthly parquet files, filtering out orphaned comments.

In [8]:
for month in months:
    print(f"\nProcessing comments for {month}...")
    print("=" * 80)
    
    # Load submissions for this month to get valid submission_ids
    submissions_file = gold_submissions_dir / f'{month}.parquet'
    if not submissions_file.exists():
        print(f"⚠️  Submissions file not found: {submissions_file.name}")
        continue
    
    df_subs = pd.read_parquet(submissions_file)
    valid_submission_ids = set(df_subs['submission_id'])
    print(f"Loaded {len(valid_submission_ids):,} valid submission IDs")
    
    # Find all daily comment files for this month
    daily_files = sorted(comments_silver_dir.glob(f'{month}-*.parquet'))
    print(f"Found {len(daily_files)} daily files")
    
    if not daily_files:
        print(f"⚠️  No files found for {month}")
        continue
    
    # Load and concatenate
    dfs = []
    for f in tqdm(daily_files, desc=f"Loading {month}"):
        df = pd.read_parquet(f)
        dfs.append(df)
    
    df_month = pd.concat(dfs, ignore_index=True)
    print(f"Loaded {len(df_month):,} comments")
    
    # Extract submission_id from link_id (remove t3_ prefix)
    df_month['submission_id'] = df_month['link_id'].str.replace('t3_', '', regex=False)
    
    # Filter out comments without matching submissions (deleted/orphaned)
    before_filter = len(df_month)
    df_month = df_month[df_month['submission_id'].isin(valid_submission_ids)]
    after_filter = len(df_month)
    dropped = before_filter - after_filter
    
    print(f"Filtered orphaned comments: dropped {dropped:,} ({100*dropped/before_filter:.1f}%)")
    print(f"Remaining: {after_filter:,} comments with valid submissions")
    
    # Apply clean schema (no author for GDPR compliance)
    df_gold = df_month[[
        'comment_id',
        'submission_id',
        'parent_id',
        'created_utc',
        'subreddit_id',
        'subreddit',
        'body'
    ]].copy()
    
    # Save monthly file
    output_file = gold_comments_dir / f'{month}.parquet'
    df_gold.to_parquet(output_file, compression='snappy', index=False)
    
    print(f"\n✓ Saved: {output_file.name}")
    print(f"  Columns: {list(df_gold.columns)}")
    print(f"  Rows: {len(df_gold):,}")

print("\n" + "=" * 80)
print("✓ Comments processing complete")


Processing comments for 2016-09...
Loaded 386,214 valid submission IDs
Found 30 daily files


Loading 2016-09: 100%|██████████| 30/30 [00:02<00:00, 12.73it/s]


Loaded 3,812,645 comments
Filtered orphaned comments: dropped 46,512 (1.2%)
Remaining: 3,766,133 comments with valid submissions

✓ Saved: 2016-09.parquet
  Columns: ['comment_id', 'submission_id', 'parent_id', 'created_utc', 'subreddit_id', 'subreddit', 'body']
  Rows: 3,766,133

Processing comments for 2016-10...
Loaded 537,217 valid submission IDs
Found 31 daily files


Loading 2016-10: 100%|██████████| 31/31 [00:02<00:00, 10.84it/s]


Loaded 4,973,150 comments
Filtered orphaned comments: dropped 40,360 (0.8%)
Remaining: 4,932,790 comments with valid submissions

✓ Saved: 2016-10.parquet
  Columns: ['comment_id', 'submission_id', 'parent_id', 'created_utc', 'subreddit_id', 'subreddit', 'body']
  Rows: 4,932,790

✓ Comments processing complete


## 5. Verify Output and Coverage

Check that submissions and comments are properly linked.

In [9]:
print("\nVerifying submission-comment linkage...")
print("=" * 80)

for month in months:
    submissions_file = gold_submissions_dir / f'{month}.parquet'
    comments_file = gold_comments_dir / f'{month}.parquet'
    
    if not submissions_file.exists() or not comments_file.exists():
        print(f"⚠️  Missing files for {month}")
        continue
    
    df_sub = pd.read_parquet(submissions_file)
    df_com = pd.read_parquet(comments_file)
    
    # Check coverage
    submission_ids_from_subs = set(df_sub['submission_id'])
    submission_ids_from_coms = set(df_com['submission_id'])
    
    coverage = len(submission_ids_from_coms & submission_ids_from_subs)
    missing = len(submission_ids_from_coms - submission_ids_from_subs)
    
    print(f"\n{month}:")
    print(f"  Submissions: {len(df_sub):,}")
    print(f"  Comments: {len(df_com):,}")
    print(f"  Unique submission_ids in comments: {len(submission_ids_from_coms):,}")
    print(f"  Matching submissions: {coverage:,} ({100*coverage/len(submission_ids_from_coms):.1f}%)")
    if missing > 0:
        print(f"  ⚠️  Comments with missing submissions: {missing:,}")
    
print("\n" + "=" * 80)
print("✓ Verification complete")
print(f"\nFinal output:")
print(f"  {gold_submissions_dir.relative_to(workspace_root)}/")
print(f"    - 2016-09.parquet")
print(f"    - 2016-10.parquet")
print(f"  {gold_comments_dir.relative_to(workspace_root)}/")
print(f"    - 2016-09.parquet")
print(f"    - 2016-10.parquet")


Verifying submission-comment linkage...

2016-09:
  Submissions: 386,214
  Comments: 3,766,133
  Unique submission_ids in comments: 206,491
  Matching submissions: 206,491 (100.0%)

2016-10:
  Submissions: 537,217
  Comments: 4,932,790
  Unique submission_ids in comments: 295,478
  Matching submissions: 295,478 (100.0%)

✓ Verification complete

Final output:
  data/01_corpus/02_gold/reddit/submissions/
    - 2016-09.parquet
    - 2016-10.parquet
  data/01_corpus/02_gold/reddit/comments/
    - 2016-09.parquet
    - 2016-10.parquet
